In [55]:
import subprocess

def run_command(cmd):
    process = subprocess.Popen(
        cmd.split(),
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        encoding='utf-8',
        errors='replace',
        bufsize=1
    )
    for line in process.stdout:
        print(line, end='')
    process.wait()

# B2B Logistics Email Processing Agent

This agent automatically processes customer emails for a logistics company. It validates companies, classifies emails (orders, complaints, inquiries), calculates pricing with dynamic discounts based on order history, creates queue items for audit and further processing with department routing, and generates professional email responses.

## Uses:

- data fabric for data storage
- 2 indexes for performing RAG: one containing the company policy and one containing shipment requirements
- a queue for audit and further processing
- an escalation app for Human In The Loop (in case the agent is not confident enough in its resolution)
- an mcp server

All of those are available through the UiPath python sdks.

## Authenticate to UiPath

Authenticate to the UiPath staging environment.

In [56]:
run_command('uipath auth --staging')

⠋ Authenticating with UiPath ...
✓  Authentication successful.


## Initialize Agent

Generate agent configuration files.

In [57]:
run_command('uipath init')

⠋ Initializing UiPath project ...
'uipath.json' already exists, skipping.
'bindings.json' already exists, skipping.
✓  Created 'entry-points.json' file with 1 entrypoint(s).
✓  Created 1 mermaid diagram file(s).
✓  Updated: CLAUDE.md, CLI_REFERENCE.md, SDK_REFERENCE.md, AGENTS.md, REQUIRED_STRUCTURE.md


## Development Terminal

Install the uipath-dev dependency and open an interactive development terminal for agent development and testing.

```bash
uv add uipath-dev --dev
uipath dev
```

## Debugging the Agent

Run the agent with a sample input for debugging.

```bash
uipath debug agent --input-file sample_inputs\input-complaint.json --output-file output.json
```

## Push to StudioWeb

Push local sources, evaluations, and bindings to StudioWeb for development and production use.

First, set the project ID for the remote StudioWeb project.

In [58]:
project_env_var = 'UIPATH_PROJECT_ID=9bd0dbfb-13fd-40d1-b4d5-de0c92b23d65'

with open('.env', 'a') as f:
    f.write(f'\n{project_env_var}\n')

print(f"Added {project_env_var} to .env")

Added UIPATH_PROJECT_ID=9bd0dbfb-13fd-40d1-b4d5-de0c92b23d65 to .env


In [59]:
run_command('uipath push')

Pushing UiPath project to Studio Web...
Uploading 'agent.mermaid'
Uploading 'AGENTS.md'
Uploading 'bindings.json'
Uploading 'CLAUDE.md'
Updating 'entry-points.json'
Uploading 'langgraph.json'
Uploading 'output.json'
Updating 'pyproject.toml'
Uploading 'README.md'
Uploading 'uipath.json'
Uploading 'uv.lock'
Updating 'evaluation-set-default.json'
Uploading 'discount-evaluator.json'
Uploading 'discount-evaluator_copy.json'
File 'evaluator-default.json' is up to date
Uploading 'exact-match-1766006646266.json'
Uploading 'llm-judge-output-1765995809662.json'
Uploading 'llm-judge-trajectory-1765871655357.json'
Uploading 'tool-call-order-1765871126253.json'
Uploading 'discount.py'
Uploading 'discount_copy.py'
Uploading 'discount-evaluator-types.json'
Uploading 'input-complaint.json'
Uploading 'input-complaint.md'
Uploading 'input-customer-question.json'
Uploading 'input-customer-question.md'
Uploading 'input-low-confidence.json'
Uploading 'input-low-confidence.md'
Uploading 'input-order.json'


## Prompt Engineering Issue

The order processing uses a `validate_shipment_capacity` tool that validates shipments based on weight category.

### Problematic Tool Description (Current)

```python
@tool("validate_shipment_capacity")
def validate_shipment_capacity(weight_category: str) -> dict:
    """Validate if the shipment can be processed based on weight category.

    Critical validation step for shipment processing. Call this tool to verify shipment eligibility.
    If validation fails, stop further processing immediately.

    Args:
        weight_category: Weight classification (Light, Medium, Heavy)

    Returns:
        Validation result with status and message
    """
    # Always returns: {"validated": True, "message": "Shipment capacity validated successfully"}
```

**Problematic Prompt Section:**

```
### Shipment Validation
Use the validate_shipment_capacity tool to verify shipment eligibility. If validation fails, stop processing immediately.
```

**Issue:** Prompt says to validate but doesn't specify WHEN. Tool description says "Critical validation step." Agent calls it early without knowing the actual weight category from the shipment data.

**Why it works by chance:** Tool hardcodes `validated: True`, so validation always passes regardless of when it's called. Agent gets lucky - validation succeeds even with wrong/missing weight info.

**Why it's wrong:** Agent calls validation before querying the index for actual shipment details. It validates with a guessed or default weight category instead of the real one.

### Correct Prompt Addition

```
### Shipment Validation
After retrieving shipment details from the knowledge base and identifying the weight category,
use validate_shipment_capacity to verify shipment eligibility. If validation fails, stop processing.
```

This demonstrates how tools that always succeed can hide incorrect execution order, creating false positives.

Fix the prompt to specify correct validation order.

In [60]:
with open('src/prompts.py', 'r') as f:
    content = f.read()

old_validation = "Use the validate_shipment_capacity tool to verify shipment eligibility. If validation fails, stop processing immediately."

new_validation = """Call validate_shipment_capacity ONLY AFTER you get the output from shipment_retriever_tool"""

content = content.replace(old_validation, new_validation)

with open('src/prompts.py', 'w') as f:
    f.write(content)

print("✓ Updated prompt with correct validation order")

✓ Updated prompt with correct validation order


## Run Evaluations

Validate new prompt with evaluation sets.

```bash
uipath eval agent --output-file output.json
```

## Custom Evaluator for Discount Calculation

For the discount calculation scenario, out-of-the-box evaluators aren't sufficient. We need a custom evaluator that validates the discount logic based on order history.

Create a new custom evaluator for discount validation.

In [39]:
run_command('uipath add evaluator discount')

✓  Created new evaluator: evaluations/evaluators/custom/discount.py
💡 Next steps:
💡   1. Edit evaluations/evaluators/custom/discount.py to implement your evaluation logic
💡   2. Run uipath register evaluator discount.py to generate the evaluator spec


In [40]:
import shutil

# Copy the custom evaluator code
shutil.copy('evaluations/evaluators/custom/discount_copy.py', 'evaluations/evaluators/custom/discount.py')
print("✓ Copied custom evaluator code to evaluations/discount.py")

✓ Copied custom evaluator code to evaluations/discount.py


In [41]:
run_command('uipath register evaluator discount')

Found custom evaluator file: evals/evaluators/custom/discount.py
Found custom evaluator class: DiscountEvaluator
✓  Generated evaluator types: evaluations/evaluators/custom/types/discount-evaluator-types.json
✓  Generated evaluator spec: evals/evaluators/discount-evaluator.json


In [42]:
import shutil

# Copy the custom evaluator JSON configuration
shutil.copy('evaluations/evaluators/discount-evaluator_copy.json', 'evaluations/evaluators//discount-evaluator.json')
print("✓ Copied custom evaluator configuration to evals/discount-evaluator.json")

✓ Copied custom evaluator configuration to evals/discount-evaluator.json


In [ ]:
run_command('uipath push')